In [1]:
from models import DetectSpan
import pandas as pd
from utils import pad, filter_and_pad_outputs
import numpy as np
import torch.nn as nn
import pickle
import torch

In [2]:
def mul(x):
    y = 1 
    for i in x:
        y *= i
    return y

In [3]:
df = pd.read_excel("final_complaints.xlsx")

In [4]:
def convert(x):
    try:
        x = [
            float(
                y.\
                strip(' ').\
                replace(':','.').\
                replace(';','.').\
                replace('-','.')
            ) 
            for y in x.strip(':. ').split('to' if 'to' in x else '-')
        ] if type(x)==str else x
    except Exception as e:
        print(e,x.count('.'),x)
        x = None
    return x
df['Timing_1'] = df['Timing_1'].apply(convert)
df['Timing_2'] = df['Timing_2'].apply(convert)
df['Timing_3'] = df['Timing_3'].apply(convert)

could not convert string to float: '0.25.2.30' 2 0.25:2.30
could not convert string to float: 'o 0.44' 0 0:21 too 0:44
could not convert string to float: '' 0 0:09 to 


In [5]:
time_cols = ['Timing_1','Timing_2','Timing_3']
for col in time_cols:
    df[col] = df[col].apply(
        lambda x:
            [
                int(x[0])*60+( x[0]-int(x[0]) )*60,
                int(x[1])*60+( x[1]-int(x[1]) )*60
            ]
            if type(x) is list and x and x is not np.nan else [0,0]
    )

In [5]:
# with open("data.pkl",'rb') as f:
#     dataset = pickle.load(f)

In [6]:
# new_dataset = []

# for ind,( key, value) in enumerate(dataset.items()):
#     print(f"\r {ind}/ {len(dataset)}",end =  ' ')
#     total_time = value[0].shape[0] 
#     start = 0
#     labels = []
#     while start < total_time:
#         label_found = False
#         try:
#             for timings in df[df["Video ID"] == key[:-4]][time_cols].iloc[0].values:
#                 if timings and timings is not np.nan and type(timings) is list and curr_time in range(int(timings[0]), int(timings[1])):
#                     labels.append(1)
#                     label_found = True
#         except:
#             pass
#         if not label_found:
#             labels.append(0)
#         start += 1
#     new_dataset.append([[x.cpu() for x in value], labels,df[df["Video ID"] == key[:-4]][time_cols]])

In [7]:
with open('dataset_with_labels.pkl','rb') as f:
    dataset = pickle.load(f)

In [8]:
span_detector = DetectSpan(512)

In [9]:
span_detector = span_detector.cuda()

In [10]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(span_detector.parameters(),lr = 0.001)

In [11]:
span_detector.training

True

In [12]:
batch_size = 12
epochs = 10
loss_cache = []
for epoch in range(epochs):
    for index in range(0, len(dataset), batch_size):
        video_features = [x[0][0] for x in dataset[index : index + batch_size]]
        audio_features = [x[0][1].squeeze(1) for x in dataset[index : index + batch_size]]
        outputs = [x[1] for x in dataset[index : index + batch_size]]
        video_features, mask = pad(video_features)
        audio_features, mask = pad(audio_features)
        features = video_features + audio_features
        outputs = filter_and_pad_outputs(outputs)
        pred = span_detector(features.cuda(), attention_mask=mask.cuda())
        l = loss(pred.view(-1, 3), outputs.view(-1).cuda())
        optimizer.zero_grad()
        print(f"\rLoss = {l.item()} ")
        loss_cache.append(l.item())
        l.backward()
        optimizer.step()


Loss = 1.0897810459136963 
Loss = 0.5543214082717896 
Loss = 0.5519925355911255 
Loss = 0.551717221736908 
Loss = 0.5516787171363831 
Loss = 0.5516864061355591 
Loss = 0.5516592264175415 
Loss = 0.5516239404678345 
Loss = 0.5516027808189392 
Loss = 0.5515934228897095 
Loss = 0.5515871047973633 
Loss = 0.5515790581703186 
Loss = 0.5515691041946411 
Loss = 0.5515577793121338 
Loss = 0.5515464544296265 
Loss = 0.5515367388725281 
Loss = 0.5515297651290894 
Loss = 0.5515246391296387 
Loss = 0.5515213012695312 
Loss = 0.551519513130188 
Loss = 0.551517128944397 
Loss = 0.5515145659446716 
Loss = 0.5515120029449463 
Loss = 0.551509439945221 
Loss = 0.551506757736206 
Loss = 0.5515050292015076 
Loss = 0.5515037178993225 
Loss = 0.5515026450157166 
Loss = 0.5515013337135315 
Loss = 0.5515000224113464 
Loss = 0.5514986515045166 
Loss = 0.5514973998069763 
Loss = 0.5514960885047913 
Loss = 0.5514950752258301 
Loss = 0.5514942407608032 
Loss = 0.5514934659004211 
Loss = 0.5514926910400391 
Loss =

In [15]:
torch.argmax(pred,axis =-1 )

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')